In [60]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 25)

In [61]:
seasons = pd.read_csv('Data/nba_player_stats_by_season.csv', index_col = 'id')
# careers = pd.read_csv('Data/nba_player_stats_by_career.csv', index_col = 'id')

# Create a few extra columns
def get_min_age(player_id):
    return seasons[seasons['player_id'] == player_id]['age'].min()

seasons['min_age'] = seasons['player_id'].apply(get_min_age)
# careers['min_age'] = careers['player_id'].apply(get_min_age)

# remove seasons with less than 100 minutes (~5 mpg)
seasons.dropna(subset=['minutes_played'], inplace = True) # removes 609 seasons where the player didn't play at all
seasons = seasons[seasons['minutes_played'] >= 100]

def get_total_seasons(player_id):
    return seasons[seasons['player_id'] == player_id].shape[0]

seasons['total_seasons'] = seasons['player_id'].apply(get_total_seasons)

def index_seasons(player_id, year):
    season_list = list(seasons[seasons['player_id'] == player_id]['year'])
    season_list.sort()
    return season_list.index(year)

seasons['season_index'] = [index_seasons(player_id, year) for player_id,
                           year in zip(seasons['player_id'], seasons['year'])]

In [62]:
# convert minutes played to % of available minutes played
# helps to account for seasons shortened by injury or lockout

seasons['minutes_proportion'] = seasons['minutes_played'] / (seasons['games_played'] * 40)

In [63]:
seasons['three_point_percentage'].fillna(value = 0.15, inplace = True)

def fix_low_3pa(attempts, rate):
    if attempts <= 5:
        return 0.15
    elif attempts <= 15:
        if rate >= 0.25:
            return 0.25
        else:
            return np.max([0.12, rate])
    elif attempts <= 25:
        if rate >= 0.35:
            return 0.35
        else:
            return np.max([0.1, rate])
    else:
        return rate

seasons['three_point_percentage'] = [fix_low_3pa(attempts, rate) for attempts,
                                  rate in zip(seasons['three_point_attempts'], seasons['three_point_percentage'])]

def fix_low_fta(attempts, rate):
    if attempts <= 10:
        if rate >= 0.75:
            return 0.75
        else:
            return np.max([0.55, rate])
    elif attempts <= 25:
        if rate >= 0.8:
            return 0.8
        else:
            return np.max([0.5, rate])
    else:
        return rate

seasons['free_throw_percentage'] = [fix_low_fta(attempts, rate) for attempts,
                                  rate in zip(seasons['free_throw_attempts'], seasons['free_throw_percentage'])]

def fix_low_2pa(attempts, rate):
    if attempts <= 25:
        if rate >= 0.6:
            return 0.6
        else:
            return np.max([0.3, rate])
    else:
        return rate

seasons['two_point_percentage'] = [fix_low_fta(attempts, rate) for attempts,
                                  rate in zip(seasons['two_point_attempts'], seasons['two_point_percentage'])]

In [64]:
# convert counting stats to per 36 minute
seasons['two_point_attempts'] = seasons['two_point_attempts'] / seasons['minutes_played'] * 36
seasons['three_point_attempts'] = seasons['three_point_attempts'] / seasons['minutes_played'] * 36
seasons['free_throw_attempts'] = seasons['free_throw_attempts'] / seasons['minutes_played'] * 36
seasons['defensive_rebounds'] = seasons['defensive_rebounds'] / seasons['minutes_played'] * 36
seasons['offensive_rebounds'] = seasons['offensive_rebounds'] / seasons['minutes_played'] * 36
seasons['assists'] = seasons['assists'] / seasons['minutes_played'] * 36
seasons['steals'] = seasons['steals'] / seasons['minutes_played'] * 36
seasons['blocks'] = seasons['blocks'] / seasons['minutes_played'] * 36
seasons['turnovers'] = seasons['turnovers'] / seasons['minutes_played'] * 36
seasons['personal_fouls'] = seasons['personal_fouls'] / seasons['minutes_played'] * 36

In [65]:
seasons['three_point_percentage'] = seasons['three_point_percentage'] * 1.8

In [66]:
# convert proportions to log odds using logit function
# define sigmoid function for reverse transformation

def logit(p):
    if p > 0.95:
        p = 0.95
    if p < 0.05:
        p = 0.05
    return np.log(p/(1-p))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [67]:
seasons['minutes_proportion'] = seasons['minutes_proportion'].apply(logit)
seasons['two_point_percentage'] = seasons['two_point_percentage'].apply(logit)
seasons['three_point_percentage'] = seasons['three_point_percentage'].apply(logit)
seasons['free_throw_percentage'] = seasons['free_throw_percentage'].apply(logit)

In [68]:
seasons['age2'] = seasons['age'] ** 2
seasons['age3'] = seasons['age'] ** 3

# Import Models

In [69]:
class MultiModelContainer:
    
    '''
    Object to function as a container for multiple linear models.
    Initialize with a dictionary where keys are targets and values are lists of features.
    '''
    
    def __init__(self, feature_dict, data):
        
        self.model_dict = {}
        self.feature_dict = feature_dict
        
        # initialize and fit models
        for target in feature_dict.keys():
            features = feature_dict[target]
            self.model_dict[target] = LinearRegression().fit(data[features], data[target])    

    def predict(self, targets, data, output_df = False, input_df = True, feature_map = None):
        
        switch = False
        
        for target in targets:
            # select features for target
            features = self.feature_dict[target]
            
            if input_df:
                # select model and input data for prediction
                preds = self.model_dict[target].predict(data[features])
            else:
                # if not using a df use feature map to select data from an array
                X = np.hstack(tuple([data[:,feature_map[feature]] for feature in features]))
                preds = self.model_dict[target].predict(X.reshape(-1,len(features)))
            
            if not switch:
                predictions = preds.reshape(-1,1)
                switch = True
                
            else:
                predictions = np.hstack((predictions, preds.reshape(-1,1)))
        
        if not output_df:
            return predictions
        
        else:
            return pd.DataFrame(predictions, columns = targets, index = data.index)

In [70]:
import pickle

with open('Models/FinalModel.pkl', 'rb') as file:
    FinalModel = pickle.load(file)
    
with open('Models/MinutesModel.pkl', 'rb') as file:
    MinutesModel = pickle.load(file)

In [71]:
features = ['age', 'two_point_percentage', 'two_point_attempts', 'three_point_percentage',
       'three_point_attempts', 'free_throw_percentage', 'free_throw_attempts',
       'defensive_rebounds', 'offensive_rebounds', 'assists', 'steals',
       'blocks', 'turnovers', 'personal_fouls', 'age2', 'age3']

# map p1 features to column index of current version
p1_features = ['age', 'two_point_percentage_p1', 'two_point_attempts_p1', 'three_point_percentage_p1',
       'three_point_attempts_p1', 'free_throw_percentage_p1', 'free_throw_attempts_p1',
       'defensive_rebounds_p1', 'offensive_rebounds_p1', 'assists_p1', 'steals_p1',
       'blocks_p1', 'turnovers_p1', 'personal_fouls_p1', 'age2', 'age3']

feature_map = {}

for feat in p1_features:
    feature_map[feat] = p1_features.index(feat)
    
targets = ['two_point_percentage', 'two_point_attempts', 'three_point_percentage',
       'three_point_attempts', 'free_throw_percentage', 'free_throw_attempts',
       'defensive_rebounds', 'offensive_rebounds', 'assists', 'steals',
       'blocks', 'turnovers', 'personal_fouls']

# Predict Future NBA Statistical Leaders

### Make predictions for every player less than or equal to age 32 for 2020 season

In [72]:
mask = np.array(seasons['year'] == 2020) & np.array(seasons['age'] <= 32)
new_players = seasons[mask]

In [73]:
print(list(new_players['player_id']))

['klebima01', 'wrighde01', 'finnedo01', 'hardati02', 'curryse01', 'jacksju01', 'brunsja01', 'doncilu01', 'porzikr01', 'poweldw01', 'marjabo01', 'caulewi01', 'kiddgmi01', 'lopezbr01', 'connapa01', 'lopezro01', 'divindo01', 'antetgi01', 'bledser01', 'middlkh01', 'brownst02', 'wilsodj01', 'mclembe01', 'hardeja01', 'riverau01', 'houseda01', 'westbru01', 'gordoer01', 'clemoch01', 'harteis01', 'covinro01', 'cabocbr01', 'simonan01', 'mccolcj01', 'whiteha01', 'lillada01', 'trentga02', 'littlna01', 'hezonma01', 'hoodro01', 'swanica01', 'gabriwe01', 'hoardja01', 'huntede01', 'youngtr01', 'reddica01', 'huertke01', 'fernabr01', 'jonesda03', 'bembrde01', 'collijo01', 'goodwbr01', 'teaguje01', 'grahatr01', 'dedmode01', 'capelca01', 'labissk01', 'hartjo01', 'ingrabr01', 'balllo01', 'hayesja02', 'holidjr01', 'mellini01', 'mooreet01', 'jacksfr01', 'favorde01', 'alexani01', 'willike04', 'okafoja01', 'willizi01', 'zubaciv01', 'harremo01', 'greenja01', 'leonaka01', 'pattepa01', 'mcgruro01', 'beverpa01', '

In [74]:
new_players.shape

(384, 99)

# Define Function for Making Future Predictions

In [75]:
def predict_career(player_id, n_years, start_year, stats_model, minutes_model):
    
    # get the players actual data and create seed for predictions
    mask = np.array(seasons['player_id'] == player_id) & np.array(seasons['year'] == start_year)
    seed = np.array(seasons[mask][features]).reshape(1,-1)
    mseed = np.array(seasons[mask]['minutes_proportion']).reshape(-1,1)
    
    # iterate through total seasons and generate prediction based on each past prediction
    for i in range(0, n_years):
        
        # get inputs for prediction, previous season stats and CURRENT age
        pred_inputs = seed[-1,1:-2].reshape(1,-1)
        current_age = np.array(seed[i,0] + 1).reshape(1,-1)
        pred_inputs = np.hstack((current_age, pred_inputs, current_age**2, current_age**3))
        preds = stats_model.predict(targets, pred_inputs, input_df = False, feature_map = feature_map)
        
        # append age features to prediction
        preds = np.hstack((current_age, preds, current_age**2, current_age**3))
        
        # append predictions to seed
        seed = np.vstack((seed, preds))
    
    # predict minutes in each season
    for i in range(0, n_years):
        pred_inputs = np.hstack((seed[i+1,:], mseed[-1,:])).reshape(1,-1)
        minutes = minutes_model.predict(pred_inputs)
        mseed = np.vstack((mseed, minutes.reshape(-1,1)))
        
    seed = np.hstack((mseed, seed))
    
    # Convert logits back to percentages
    seed[:,0] = sigmoid(seed[:,0])
    
    for i, feat in enumerate(features):
        if feat[-10:] == 'percentage':
            seed[:,i + 1] = sigmoid(seed[:,i + 1])
    
    # Remove age2 and age3 features
    seed = seed[:,:-2]
    
    # Output Formatting
    cols = ['minutes_proportion'] + features[:-2]
    seed = pd.DataFrame(np.round(seed, 3), columns = cols)
    seed['player_id'] = seasons[mask]['player_id'][0]
    seed['name'] = seasons[mask]['name'][0]
    seed['year'] = [start_year + i for i in range(n_years + 1)]
    seed['index_col'] = [player_id + str(year) for player_id, year in zip(seed['player_id'], seed['year'])]
    seed.set_index('index_col', inplace = True)
    
    # Adjust 3pt percentage by dividing by 1.8
    seed['three_point_percentage'] = seed['three_point_percentage'] / 1.8
    
    #order columns for output
    final_cols = ['player_id', 'name', 'year'] + cols
    
    return seed[final_cols]

In [76]:
career_preds = {}

for player_id in list(new_players['player_id']):
    career_preds[player_id] = predict_career(player_id, 10, 2020, FinalModel, MinutesModel)

# Define Function to Convert to Per Game Averages

In [77]:
from copy import deepcopy

In [78]:
def per_game_averages(player_df):
    pg = deepcopy(player_df)
    
    pg['minutes_per_game'] = pg['minutes_proportion'] * 40
    pg.drop(columns = 'minutes_proportion', inplace = True)
    
    pg['two_pointers'] = pg['two_point_attempts'] * pg['two_point_percentage'] * pg['minutes_per_game'] / 36
    
    pg['three_pointers'] = pg['three_point_attempts'] * pg['three_point_percentage'] * pg['minutes_per_game'] / 36
    
    pg['free_throws'] = pg['free_throw_attempts'] * pg['free_throw_percentage'] * pg['minutes_per_game'] / 36

    pg['defensive_rebounds'] = pg['defensive_rebounds'] * pg['minutes_per_game'] / 36
    
    pg['offensive_rebounds'] = pg['offensive_rebounds'] * pg['minutes_per_game'] / 36
    
    pg['total_rebounds'] = pg['defensive_rebounds'] + pg['offensive_rebounds']
    
    pg['assists'] = pg['assists'] * pg['minutes_per_game'] / 36
    
    pg['steals'] = pg['steals'] * pg['minutes_per_game'] / 36
    
    pg['blocks'] = pg['blocks'] * pg['minutes_per_game'] / 36
    
    pg['turnovers'] = pg['turnovers'] * pg['minutes_per_game'] / 36
    
    pg['personal_fouls'] = pg['personal_fouls'] * pg['minutes_per_game'] / 36
    
    pg['points'] = 2 * pg['two_pointers'] + 3 * pg['three_pointers'] + 1 * pg['free_throws']
    
    pg['field_goals'] = pg['two_pointers'] + pg['three_pointers']
    
    pg['field_goal_percentage'] = pg['field_goals'] / ((pg['two_point_attempts'] + pg['three_point_attempts']) * pg['minutes_per_game'] / 36)
    
    final_columns = ['player_id', 'name', 'year', 'age', 'minutes_per_game', 'field_goals', 'field_goal_percentage',
                     'three_pointers', 'three_point_percentage', 'two_pointers', 'two_point_percentage',  'free_throws',
                     'free_throw_percentage', 'total_rebounds', 'offensive_rebounds', 'defensive_rebounds',
                     'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'points']
    
    pg = pg.round(2)
    
    return pg[final_columns]

In [79]:
per_game_preds = {}

for player_id in career_preds.keys():

    per_game_preds[player_id] = per_game_averages(career_preds[player_id])

# Determine Statistical Leaders

In [80]:
# Combine all the dataframes

switch = 0

for player_id in per_game_preds.keys():
    
    if not switch:
        per_game_df = per_game_preds[player_id]
        switch = 1
    
    else:
        per_game_df = pd.concat([per_game_df, per_game_preds[player_id]], axis = 0)

In [81]:
def stat_leaders(year, stat, N):
    
    stat_df = per_game_df[per_game_df['year'] == year][['player_id', 'name', stat]]
    return stat_df.sort_values(stat, ascending = False).head(N)

In [82]:
stat_leaders(2023, 'points', 10)

,player_id,name,points
index_col,,,
doncilu012023,doncilu01,Luka Dončić,33.20
youngtr012023,youngtr01,Trae Young,31.16
antetgi012023,antetgi01,Giannis Antetokounmpo,29.95
willizi012023,willizi01,Zion Williamson,27.26
hardeja012023,hardeja01,James Harden,25.30
bealbr012023,bealbr01,Bradley Beal,24.41
tatumja012023,tatumja01,Jayson Tatum,24.22
leonaka012023,leonaka01,Kawhi Leonard,24.16
ingrabr012023,ingrabr01,Brandon Ingram,23.52


In [83]:
stat_leaders(2023, 'total_rebounds', 10)

,player_id,name,total_rebounds
index_col,,,
antetgi012023,antetgi01,Giannis Antetokounmpo,13.73
drumman012023,drumman01,Andre Drummond,12.71
whiteha012023,whiteha01,Hassan Whiteside,11.90
capelca012023,capelca01,Clint Capela,11.72
allenja012023,allenja01,Jarrett Allen,11.59
goberru012023,goberru01,Rudy Gobert,11.51
embiijo012023,embiijo01,Joel Embiid,11.44
aytonde012023,aytonde01,Deandre Ayton,11.29
valanjo012023,valanjo01,Jonas Valančiūnas,10.76


In [84]:
stat_leaders(2023, 'assists', 10)

,player_id,name,assists
index_col,,,
youngtr012023,youngtr01,Trae Young,8.80
doncilu012023,doncilu01,Luka Dončić,8.73
moranja012023,moranja01,Ja Morant,7.82
simmobe012023,simmobe01,Ben Simmons,7.29
foxde012023,foxde01,De'Aaron Fox,7.04
rubiori012023,rubiori01,Ricky Rubio,6.87
jonesty012023,jonesty01,Tyus Jones,6.87
paytoel012023,paytoel01,Elfrid Payton,6.70
mclaujo012023,mclaujo01,Jordan McLaughlin,6.38


In [85]:
stat_leaders(2023, 'steals', 10)

,player_id,name,steals
index_col,,,
simmobe012023,simmobe01,Ben Simmons,1.74
murrade012023,murrade01,Dejounte Murray,1.63
meltode012023,meltode01,De'Anthony Melton,1.58
mclaujo012023,mclaujo01,Jordan McLaughlin,1.56
foxde012023,foxde01,De'Aaron Fox,1.53
paytoel012023,paytoel01,Elfrid Payton,1.49
jonesty012023,jonesty01,Tyus Jones,1.49
doncilu012023,doncilu01,Luka Dončić,1.48
fultzma012023,fultzma01,Markelle Fultz,1.46


In [86]:
stat_leaders(2023, 'blocks', 10)

,player_id,name,blocks
index_col,,,
whiteha012023,whiteha01,Hassan Whiteside,2.21
bambamo012023,bambamo01,Mo Bamba,2.01
robinmi012023,robinmi01,Mitchell Robinson,1.99
williro042023,williro04,Robert Williams,1.99
isaacjo012023,isaacjo01,Jonathan Isaac,1.95
poeltja012023,poeltja01,Jakob Poeltl,1.90
davisan022023,davisan02,Anthony Davis,1.85
gaffoda012023,gaffoda01,Daniel Gafford,1.73
bitadgo012023,bitadgo01,Goga Bitadze,1.64


In [87]:
stat_leaders(2023, 'three_pointers', 10)

,player_id,name,three_pointers
index_col,,,
russeda012023,russeda01,D'Angelo Russell,3.86
youngtr012023,youngtr01,Trae Young,3.86
curryst012023,curryst01,Stephen Curry,3.77
hardeja012023,hardeja01,James Harden,3.76
doncilu012023,doncilu01,Luka Dončić,3.69
clemoch012023,clemoch01,Chris Clemons,3.63
lillada012023,lillada01,Damian Lillard,3.43
grahade012023,grahade01,Devonte' Graham,3.12
hieldbu012023,hieldbu01,Buddy Hield,3.05


In [88]:
stat_leaders(2023, 'free_throws', 10)

,player_id,name,free_throws
index_col,,,
doncilu012023,doncilu01,Luka Dončić,7.28
antetgi012023,antetgi01,Giannis Antetokounmpo,7.02
youngtr012023,youngtr01,Trae Young,7.00
willizi012023,willizi01,Zion Williamson,6.74
hardeja012023,hardeja01,James Harden,5.83
embiijo012023,embiijo01,Joel Embiid,5.53
davisan022023,davisan02,Anthony Davis,5.26
foxde012023,foxde01,De'Aaron Fox,5.25
bookede012023,bookede01,Devin Booker,5.06


In [89]:
stat_leaders(2023, 'minutes_per_game', 10)

,player_id,name,minutes_per_game
index_col,,,
willizi012023,willizi01,Zion Williamson,38.24
doncilu012023,doncilu01,Luka Dončić,38.20
youngtr012023,youngtr01,Trae Young,38.04
antetgi012023,antetgi01,Giannis Antetokounmpo,37.32
moranja012023,moranja01,Ja Morant,37.32
gilgesh012023,gilgesh01,Shai Gilgeous-Alexander,36.68
tatumja012023,tatumja01,Jayson Tatum,36.16
foxde012023,foxde01,De'Aaron Fox,36.04
davisan022023,davisan02,Anthony Davis,35.88


In [90]:
stat_leaders(2023, 'three_point_percentage', 10)

,player_id,name,three_point_percentage
index_col,,,
clemoch012023,clemoch01,Chris Clemons,0.46
robindu012023,robindu01,Duncan Robinson,0.45
hieldbu012023,hieldbu01,Buddy Hield,0.44
curryst012023,curryst01,Stephen Curry,0.44
russeda012023,russeda01,D'Angelo Russell,0.44
mclembe012023,mclembe01,Ben McLemore,0.43
lillada012023,lillada01,Damian Lillard,0.43
beaslma012023,beaslma01,Malik Beasley,0.43
valende012023,valende01,Denzel Valentine,0.43


In [95]:
per_game_df[np.array(per_game_df['player_id'] == 'doncilu01') & np.array(per_game_df['year'] == 2023)]

,player_id,name,year,age,minutes_per_game,field_goals,field_goal_percentage,three_pointers,three_point_percentage,two_pointers,two_point_percentage,free_throws,free_throw_percentage,total_rebounds,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
index_col,,,,,,,,,,,,,,,,,,,,,,
doncilu012023,doncilu01,Luka Dončić,2023,23.83,38.2,11.12,0.49,3.69,0.41,7.43,0.54,7.28,0.83,8.81,1.22,7.59,8.73,1.48,0.48,4.29,1.84,33.2


In [93]:
per_game_df[per_game_df['player_id'] == 'youngtr01']

,player_id,name,year,age,minutes_per_game,field_goals,field_goal_percentage,three_pointers,three_point_percentage,two_pointers,two_point_percentage,free_throws,free_throw_percentage,total_rebounds,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
index_col,,,,,,,,,,,,,,,,,,,,,,
youngtr012020,youngtr01,Trae Young,2020,21.25,35.32,9.10,0.44,3.42,0.36,5.68,0.50,8.01,0.86,4.25,0.53,3.72,9.33,1.08,0.13,4.82,1.73,29.63
youngtr012021,youngtr01,Trae Young,2021,22.25,37.16,10.02,0.46,3.81,0.41,6.21,0.50,7.84,0.86,4.42,0.36,4.06,9.38,1.27,0.18,4.60,1.62,31.68
youngtr012022,youngtr01,Trae Young,2022,23.25,37.84,10.24,0.47,3.89,0.42,6.35,0.50,7.45,0.86,4.42,0.23,4.19,9.14,1.38,0.20,4.33,1.49,31.82
youngtr012023,youngtr01,Trae Young,2023,24.25,38.04,10.15,0.47,3.86,0.43,6.29,0.50,7.00,0.86,4.33,0.13,4.19,8.80,1.44,0.21,4.06,1.42,31.16
youngtr012024,youngtr01,Trae Young,2024,25.25,37.96,9.88,0.47,3.80,0.44,6.09,0.50,6.50,0.86,4.20,0.07,4.13,8.42,1.47,0.21,3.80,1.40,30.06
youngtr012025,youngtr01,Trae Young,2025,26.25,37.76,9.51,0.47,3.71,0.44,5.80,0.49,6.00,0.86,4.06,0.03,4.04,8.04,1.47,0.20,3.57,1.42,28.74
youngtr012026,youngtr01,Trae Young,2026,27.25,37.36,9.07,0.47,3.60,0.44,5.47,0.49,5.49,0.86,3.92,0.00,3.91,7.63,1.45,0.19,3.35,1.47,27.24
youngtr012027,youngtr01,Trae Young,2027,28.25,36.84,8.57,0.46,3.48,0.43,5.09,0.48,4.99,0.86,3.77,-0.01,3.78,7.23,1.42,0.18,3.14,1.54,25.61
youngtr012028,youngtr01,Trae Young,2028,29.25,36.12,8.03,0.46,3.33,0.43,4.69,0.48,4.49,0.85,3.62,-0.01,3.63,6.80,1.37,0.16,2.93,1.61,23.88


In [94]:
per_game_df[per_game_df['player_id'] == 'antetgi01']

,player_id,name,year,age,minutes_per_game,field_goals,field_goal_percentage,three_pointers,three_point_percentage,two_pointers,two_point_percentage,free_throws,free_throw_percentage,total_rebounds,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
index_col,,,,,,,,,,,,,,,,,,,,,,
antetgi012020,antetgi01,Giannis Antetokounmpo,2020,25.0,30.92,10.92,0.55,1.45,0.31,9.47,0.62,6.33,0.63,13.73,2.28,11.45,5.77,1.03,1.02,3.67,3.03,29.63
antetgi012021,antetgi01,Giannis Antetokounmpo,2021,26.0,35.72,11.70,0.52,1.57,0.29,10.13,0.60,7.49,0.71,14.87,2.81,12.05,6.37,1.24,1.25,4.18,3.12,32.47
antetgi012022,antetgi01,Giannis Antetokounmpo,2022,27.0,37.00,11.33,0.51,1.57,0.29,9.76,0.58,7.44,0.75,14.46,2.97,11.49,6.28,1.29,1.33,4.17,3.03,31.68
antetgi012023,antetgi01,Giannis Antetokounmpo,2023,28.0,37.32,10.70,0.50,1.52,0.29,9.18,0.56,7.02,0.76,13.73,2.96,10.77,6.03,1.29,1.34,3.99,2.97,29.95
antetgi012024,antetgi01,Giannis Antetokounmpo,2024,29.0,37.12,9.96,0.49,1.46,0.29,8.51,0.55,6.45,0.77,12.89,2.88,10.02,5.69,1.26,1.32,3.76,2.95,27.83
antetgi012025,antetgi01,Giannis Antetokounmpo,2025,30.0,36.64,9.18,0.48,1.37,0.29,7.81,0.54,5.85,0.77,12.06,2.75,9.31,5.33,1.22,1.27,3.50,2.94,25.58
antetgi012026,antetgi01,Giannis Antetokounmpo,2026,31.0,35.84,8.38,0.47,1.28,0.29,7.10,0.53,5.22,0.77,11.21,2.60,8.61,4.95,1.16,1.21,3.24,2.94,23.26
antetgi012027,antetgi01,Giannis Antetokounmpo,2027,32.0,34.76,7.56,0.46,1.17,0.29,6.39,0.52,4.62,0.77,10.37,2.43,7.94,4.56,1.10,1.14,2.96,2.92,20.91
antetgi012028,antetgi01,Giannis Antetokounmpo,2028,33.0,33.36,6.76,0.46,1.06,0.28,5.69,0.52,4.03,0.77,9.52,2.25,7.28,4.16,1.04,1.06,2.69,2.88,18.61
